__author__ = 'Ricardo Pasquini'

# Counting tweets at the hexagon level. Complete procedure

Overview: This code counts georeferenced tweets at the hexagon level. Hexagons are uniquely identified in space. We use Uber's H3 library for this purpose.  
The code  also performs a home location prediction for each user. This allows later counting tweets by whether they were tweeted by residents, residents of neighbors hexagons, or non-residents.
The project was designed to be scalable to millions of tweets. The process is done with MongoDB.

By default the project generates the following collections on the Mongo database:

tweets: Collection containing tweets. This collection is modified by incorporating the corresponding hex id. \
users: Collection containing users. This collection is modified by incorporating the home location data.\
hexcounts: Collection containing counts of tweets.


In [1]:
import sys
sys.path.append("../") 
import databasepopulation
import communicationwmongo as commu
import home_location as home
import analysis as a
import pymongo
import pandas as pd

In [2]:
import importlib
importlib.reload(databasepopulation)
importlib.reload(home)
importlib.reload(a)

<module 'analysis' from '..\\analysis.py'>

# Counting tweets at the hexagon level. Complete procedure

This version of the code assumes that a different database will be used for each city.


If necessary check your current location with the following command:

In [3]:
pwd

'C:\\Users\\emman\\Documents\\git\\twitter_and_displacement\\notebooks'

# 0. Connect to Mongo and define a specific database

By default connect to Mongo running on a localhost. 
Requires manually changing the parameters if working on a cloud environment

I will use the name buenosaires as the db example. Recall each database for each city.


In [5]:
db=commu.connecttoLocaldb(database='twitter_sf')

In [7]:
db.twitter_sf

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'twitter_sf'), 'twitter_sf')

In [ ]:
twt_sf = pd.read_csv('C:/Users/emman/Box/Twitter data/North America/SANFRANCISCO_2012-2015.csv')

# 1. Raw data to Mongo
This function populates the files containing tweets into Mongo.
Assumes files are named like: ba_2012.csv, etc.
Choose start and end years.


In [7]:
databasepopulation.populatetweets(db, path='/Users/emman/Box/Twitter data/Asia/Sydney/', cityprefix='sydney', yearstart=2012, yearend=2012)

Now populating year  2012
process completed


In [8]:
db.tweets.update({},{'$rename':{"latitude":"lat","longitude":"lon"}}, False,True,True)

C:\Users\emman\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  """Entry point for launching an IPython kernel.


{'n': 6043344, 'nModified': 6043344, 'ok': 1.0, 'updatedExisting': True}

# 2. Adding Hexids to tweets 
This function includes hexagons ids to each tweet.  

In [9]:
databasepopulation.addhexjob(db)

 iter: 1  time: 0.6439850330352783
 iter: 2  time: 0.22062468528747559
 iter: 3  time: 0.22016096115112305
 iter: 4  time: 0.24099016189575195
 iter: 5  time: 0.22933197021484375
 iter: 6  time: 0.2373640537261963
 iter: 7  time: 0.2234032154083252
 iter: 8  time: 0.23836135864257812
 iter: 9  time: 0.253328800201416
 iter: 10  time: 0.2854783535003662
 iter: 11  time: 0.242523193359375
 iter: 12  time: 0.23321175575256348
 iter: 13  time: 0.23038315773010254
 iter: 14  time: 0.2642934322357178
 iter: 15  time: 0.24654030799865723
 iter: 16  time: 0.2323775291442871
 iter: 17  time: 0.23635053634643555
 iter: 18  time: 0.28430867195129395
 iter: 19  time: 0.2561221122741699
 iter: 20  time: 0.28822875022888184
 iter: 21  time: 0.303189754486084
 iter: 22  time: 0.39300537109375
 iter: 23  time: 0.3057219982147217
 iter: 24  time: 0.25197291374206543
 iter: 25  time: 0.2537117004394531
 iter: 26  time: 0.3532381057739258
 iter: 27  time: 0.32512879371643066
 iter: 28  time: 0.3121621608

 iter: 223  time: 0.2969658374786377
 iter: 224  time: 0.4479706287384033
 iter: 225  time: 0.439255952835083
 iter: 226  time: 0.6178295612335205
 iter: 227  time: 0.4617652893066406
 iter: 228  time: 0.49866557121276855
 iter: 229  time: 0.4081838130950928
 iter: 230  time: 0.3052692413330078
 iter: 231  time: 0.3236682415008545
 iter: 232  time: 0.3814578056335449
 iter: 233  time: 0.3262031078338623
 iter: 234  time: 0.2962000370025635
 iter: 235  time: 0.306715726852417
 iter: 236  time: 0.370647668838501
 iter: 237  time: 0.3915853500366211
 iter: 238  time: 0.3776082992553711
 iter: 239  time: 0.5294702053070068
 iter: 240  time: 0.3373837471008301
 iter: 241  time: 0.38341379165649414
 iter: 242  time: 0.36902427673339844
 iter: 243  time: 0.3849451541900635
 iter: 244  time: 0.33469414710998535
 iter: 245  time: 0.2999858856201172
 iter: 246  time: 0.3393137454986572
 iter: 247  time: 0.32714247703552246
 iter: 248  time: 0.3451063632965088
 iter: 249  time: 0.3890311717987060

 iter: 443  time: 0.3276371955871582
 iter: 444  time: 0.28717613220214844
 iter: 445  time: 0.28913450241088867
 iter: 446  time: 0.27330780029296875
 iter: 447  time: 0.282839298248291
 iter: 448  time: 0.2613074779510498
 iter: 449  time: 0.2577972412109375
 iter: 450  time: 0.261777400970459
 iter: 451  time: 0.2872638702392578
 iter: 452  time: 0.27564334869384766
 iter: 453  time: 0.30785179138183594
 iter: 454  time: 0.3037407398223877
 iter: 455  time: 0.3387331962585449
 iter: 456  time: 0.33049917221069336
 iter: 457  time: 0.30639052391052246
 iter: 458  time: 0.2763173580169678
 iter: 459  time: 0.2674415111541748
 iter: 460  time: 0.2838249206542969
 iter: 461  time: 0.26117539405822754
 iter: 462  time: 0.2910032272338867
 iter: 463  time: 0.3221592903137207
 iter: 464  time: 0.30518198013305664
 iter: 465  time: 0.3450794219970703
 iter: 466  time: 0.3086221218109131
 iter: 467  time: 0.3303399085998535
 iter: 468  time: 0.30914759635925293
 iter: 469  time: 0.3381655216

 iter: 663  time: 0.3049039840698242
 iter: 664  time: 0.2763700485229492
 iter: 665  time: 0.26534390449523926
 iter: 666  time: 0.26790809631347656
 iter: 667  time: 0.26061153411865234
 iter: 668  time: 0.25842928886413574
 iter: 669  time: 0.26482391357421875
 iter: 670  time: 0.29903149604797363
 iter: 671  time: 0.2835111618041992
 iter: 672  time: 0.27187180519104004
 iter: 673  time: 0.2882838249206543
 iter: 674  time: 0.2808198928833008
 iter: 675  time: 0.27428555488586426
 iter: 676  time: 0.26560139656066895
 iter: 677  time: 0.25748300552368164
 iter: 678  time: 0.2952721118927002
 iter: 679  time: 0.2543354034423828
 iter: 680  time: 0.267392635345459
 iter: 681  time: 0.28423619270324707
 iter: 682  time: 0.296297550201416
 iter: 683  time: 0.27440643310546875
 iter: 684  time: 0.36426758766174316
 iter: 685  time: 0.5046536922454834
 iter: 686  time: 0.40491485595703125
 iter: 687  time: 0.35396838188171387
 iter: 688  time: 0.32836103439331055
 iter: 689  time: 0.2929

 iter: 882  time: 0.32915234565734863
 iter: 883  time: 0.3700103759765625
 iter: 884  time: 0.39394640922546387
 iter: 885  time: 0.31114745140075684
 iter: 886  time: 0.3257873058319092
 iter: 887  time: 0.3604288101196289
 iter: 888  time: 0.2842402458190918
 iter: 889  time: 0.27027392387390137
 iter: 890  time: 0.2703087329864502
 iter: 891  time: 0.26221132278442383
 iter: 892  time: 0.2972428798675537
 iter: 893  time: 0.25232911109924316
 iter: 894  time: 0.2492983341217041
 iter: 895  time: 0.256866455078125
 iter: 896  time: 0.2648732662200928
 iter: 897  time: 0.26334095001220703
 iter: 898  time: 0.25796031951904297
 iter: 899  time: 0.2523970603942871
 iter: 900  time: 0.25136375427246094
 iter: 901  time: 0.29122138023376465
 iter: 902  time: 0.2524423599243164
 iter: 903  time: 0.2403569221496582
 iter: 904  time: 0.24533987045288086
 iter: 905  time: 0.2511451244354248
 iter: 906  time: 0.2552797794342041
 iter: 907  time: 0.24933290481567383
 iter: 908  time: 0.2566285

...

 iter: 5170  time: 0.4259035587310791
 iter: 5171  time: 0.4297051429748535
 iter: 5172  time: 0.4827084541320801
 iter: 5173  time: 0.44305920600891113
 iter: 5174  time: 0.5203676223754883
 iter: 5175  time: 0.4185192584991455
 iter: 5176  time: 0.4388291835784912
 iter: 5177  time: 0.40966200828552246
 iter: 5178  time: 0.42835211753845215
 iter: 5179  time: 0.4122731685638428
 iter: 5180  time: 0.4497678279876709
 iter: 5181  time: 0.5382370948791504
 iter: 5182  time: 0.6863172054290771
 iter: 5183  time: 0.5016567707061768
 iter: 5184  time: 0.45540833473205566
 iter: 5185  time: 0.47578930854797363
 iter: 5186  time: 0.46276164054870605
 iter: 5187  time: 0.4161391258239746
 iter: 5188  time: 0.4449160099029541
 iter: 5189  time: 0.6007430553436279
 iter: 5190  time: 0.6103987693786621
 iter: 5191  time: 0.6835780143737793
 iter: 5192  time: 0.5520639419555664
 iter: 5193  time: 0.4782259464263916
 iter: 5194  time: 0.45674610137939453
 iter: 5195  time: 0.41027212142944336
 ite

 iter: 5384  time: 0.48171162605285645
 iter: 5385  time: 0.4684443473815918
 iter: 5386  time: 0.4417588710784912
 iter: 5387  time: 0.4062232971191406
 iter: 5388  time: 0.4418153762817383
 iter: 5389  time: 0.5000035762786865
 iter: 5390  time: 0.44991636276245117
 iter: 5391  time: 0.46177220344543457
 iter: 5392  time: 0.434833288192749
 iter: 5393  time: 0.4303877353668213
 iter: 5394  time: 0.4120938777923584
 iter: 5395  time: 0.46774911880493164
 iter: 5396  time: 0.42836976051330566
 iter: 5397  time: 0.5205128192901611
 iter: 5398  time: 0.4258906841278076
 iter: 5399  time: 0.4598078727722168
 iter: 5400  time: 0.4363555908203125
 iter: 5401  time: 0.47424864768981934
 iter: 5402  time: 0.5016932487487793
 iter: 5403  time: 0.44054579734802246
 iter: 5404  time: 0.45381665229797363
 iter: 5405  time: 0.47324275970458984
 iter: 5406  time: 0.4364476203918457
 iter: 5407  time: 0.44013142585754395
 iter: 5408  time: 0.4507448673248291
 iter: 5409  time: 0.426347017288208
 ite

 iter: 5598  time: 0.4530673027038574
 iter: 5599  time: 0.4323272705078125
 iter: 5600  time: 0.4805448055267334
 iter: 5601  time: 0.5271024703979492
 iter: 5602  time: 0.5046529769897461
 iter: 5603  time: 0.5119390487670898
 iter: 5604  time: 0.4898502826690674
 iter: 5605  time: 0.4812452793121338
 iter: 5606  time: 0.4764821529388428
 iter: 5607  time: 0.45190954208374023
 iter: 5608  time: 0.4323580265045166
 iter: 5609  time: 0.46575403213500977
 iter: 5610  time: 0.47733473777770996
 iter: 5611  time: 0.4759337902069092
 iter: 5612  time: 0.4677577018737793
 iter: 5613  time: 0.45163726806640625
 iter: 5614  time: 0.44284939765930176
 iter: 5615  time: 0.4269134998321533
 iter: 5616  time: 0.44876980781555176
 iter: 5617  time: 0.44467639923095703
 iter: 5618  time: 0.45232343673706055
 iter: 5619  time: 0.4408242702484131
 iter: 5620  time: 0.4862344264984131
 iter: 5621  time: 0.43767333030700684
 iter: 5622  time: 0.45592284202575684
 iter: 5623  time: 0.4448397159576416
 i

 iter: 5813  time: 0.48107147216796875
 iter: 5814  time: 0.4877030849456787
 iter: 5815  time: 0.5231149196624756
 iter: 5816  time: 0.6298279762268066
 iter: 5817  time: 0.5311334133148193
 iter: 5818  time: 0.5495645999908447
 iter: 5819  time: 0.4860706329345703
 iter: 5820  time: 0.5091581344604492
 iter: 5821  time: 0.4713151454925537
 iter: 5822  time: 0.469836950302124
 iter: 5823  time: 0.5113303661346436
 iter: 5824  time: 0.4764823913574219
 iter: 5825  time: 0.46740078926086426
 iter: 5826  time: 0.5418562889099121
 iter: 5827  time: 0.5053002834320068
 iter: 5828  time: 0.49068760871887207
 iter: 5829  time: 0.44435763359069824
 iter: 5830  time: 0.44570064544677734
 iter: 5831  time: 0.4332091808319092
 iter: 5832  time: 0.416884183883667
 iter: 5833  time: 0.41643762588500977
 iter: 5834  time: 0.49714231491088867
 iter: 5835  time: 0.4433910846710205
 iter: 5836  time: 0.42087292671203613
 iter: 5837  time: 0.4514033794403076
 iter: 5838  time: 0.45636439323425293
 iter

 iter: 6028  time: 0.5663111209869385
 iter: 6029  time: 0.5745162963867188
 iter: 6030  time: 0.5505280494689941
 iter: 6031  time: 0.5402581691741943
 iter: 6032  time: 0.5993690490722656
 iter: 6033  time: 0.5469458103179932
 iter: 6034  time: 0.5812249183654785
 iter: 6035  time: 0.4817781448364258
 iter: 6036  time: 0.476421594619751
 iter: 6037  time: 0.42398667335510254
 iter: 6038  time: 0.4527890682220459
 iter: 6039  time: 0.46651530265808105
 iter: 6040  time: 0.4857032299041748
 iter: 6041  time: 0.44167351722717285
 iter: 6042  time: 0.5196146965026855
 iter: 6043  time: 0.48175930976867676
total elapsed time: 2597.4199578762054


In [10]:
# the following is a check and a fix because I think the previous process did not run completely, probably a bug.
# are there any other documents that were not processed?
len(list(db.tweets.find({ "hex" : { "$exists" : False } })))

344

In [11]:
#Process any missing document
cursor=db.tweets.find({ "hex" : { "$exists" : False } })
df = pd.DataFrame(list(cursor))
requests = databasepopulation.add_hexs_and_prepare_bulk_request(df, dataformat='raw')
try:
    db.tweets.bulk_write(requests, ordered=False)
except BulkWriteError as bwe:
    print(bwe.details)

# 3. Database performance task: Add indexes to tweets

In [12]:
databasepopulation.create_indexes(db)

# 4. Populate Users Collection

In [13]:
databasepopulation.populate_users_collection(db)

# 5. Find Home Job for each user id in the database

In [45]:
print('Users with home location identified', db.users.count_documents({'hex9': { '$exists': True} }))


Users with home location identified 19441


# 6 Generate a Hex-level collection including all hexs in the database

By default the collection include as ids only the ids of hexagons at resolution 9
Note that many types of hexogons could coexist in a same collection because hex identifiers are unique. This is not implemented in the current version of the code, though.

In [47]:
databasepopulation.populate_hexcounts_collection(db)

# 7 Count tweets in each hex by residents and non-residents

In [68]:
import analysis as a

In [70]:
a.countandpopulatejob(db)

Hexagons pending to analyze.. 17157
iter: 50
iter: 100
iter: 150
iter: 200
iter: 250
iter: 300
iter: 350
iter: 400
iter: 450
iter: 500
iter: 550
iter: 600
iter: 650
iter: 700
iter: 750
iter: 800
iter: 850
iter: 900
iter: 950
iter: 1000
iter: 1050
iter: 1100
iter: 1150
iter: 1200
iter: 1250
iter: 1300
iter: 1350
iter: 1400
iter: 1450
iter: 1500
iter: 1550
iter: 1600
iter: 1650
iter: 1700
iter: 1750
iter: 1800
iter: 1850
iter: 1900
iter: 1950
iter: 2000
iter: 2050
iter: 2100
iter: 2150
iter: 2200
iter: 2250
iter: 2300
iter: 2350
iter: 2400
iter: 2450
iter: 2500
iter: 2550
iter: 2600
iter: 2650
iter: 2700
iter: 2750
iter: 2800
iter: 2850
iter: 2900
iter: 2950
iter: 3000
iter: 3050
iter: 3100
iter: 3150
iter: 3200
iter: 3250
iter: 3300
iter: 3350
iter: 3400
iter: 3450
iter: 3500
iter: 3550
iter: 3600
iter: 3650
iter: 3700
iter: 3750
iter: 3800
iter: 3850
iter: 3900
iter: 3950
iter: 4000
iter: 4050
iter: 4100
iter: 4150
iter: 4200
iter: 4250
iter: 4300
iter: 4350
iter: 4400
iter: 4450
iter:

# 8 Query to the DB to create the counts dataframe 

In [71]:
df=a.hexcountsresults_to_df(db, save=False)
# if save=True then saves resulting dataframe a pickle in ./hexcountsdf.pkl

In [75]:
df.describe()

,nonresidents,nonresidentsandnonneighbors,residents,totalcounts
count,204800.000000,203578.000000,32190.000000,206078.000000
mean,25.376123,23.488859,26.291208,29.325518
std,186.169569,178.511885,97.439338,196.890008
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,2.000000,2.000000,4.000000,2.000000
75%,9.000000,9.000000,18.000000,11.000000
max,26034.000000,26034.000000,4526.000000,26034.000000


In [74]:
df.to_csv('C:/Users/Emman/Desktop/Sydney_Hexes_RP.csv')